In [7]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
from ipywidgets import (
    HTML,
    Button,
    HBox
)

In [8]:
import ipywidgets as widgets
l = HTML(value='')
m = Map(center=(0,0), zoom=2)
p = Button(description='<')
n = Button(description='>')

def hover_handler(event=None, id=None, properties=None):
    l.value = properties['label']

In [9]:
def is_point(bbox):
    return (abs(bbox[0] - bbox[2]) < 0.01 and abs(bbox[1] - bbox[3]) < 0.01)

def bbox2feature(bbox, title):
    [w, s, e, n] = bbox
    feature = {
        'type': 'Feature',
        'properties': {
            'style': {'color': 'white', 'weight': 1, 'fillColor':'blue', 'fillOpacity':0.5},
            'label': title
        },
        'geometry': {
            'type': 'Polygon',
            'coordinates': [[[w,s], [e,s], [e,n], [w,n], [w,s]]]
        }
    }
    g = GeoJSON(data=feature, hover_style={'fillColor': 'red'})
    g.on_hover(hover_handler)
    return g

def bbox2marker(bbox, title):
    [w, s, e, n] = bbox
    return Marker(location=(s,w), draggable=False, title=title)

def place2layer(place):
    bbox = place['bbox']
    title = place['title']
    if is_point(bbox):
        return bbox2marker(bbox, title)
    else:
        return bbox2feature(bbox, title)

In [10]:
from IPython.display import clear_output

def labelFor(feature):
    return (
        'Spatial coverage for Pleiades period "'
        + feature['properties']['id'].split('/')[-1]
        + '": '
        + feature['properties']['label']
    )

def show(feature):
    feature['properties']['style'] = {
        'color':'red',
        'weight': 1,
        'fillColor':'blue',
        'fillOpacity':0.5
    }
    center = [
        (feature['bbox'][0] + feature['bbox'][2])/2,
        (feature['bbox'][1] + feature['bbox'][3])/2
    ]
    l.value = labelFor(feature)
    m = Map(center=center, zoom=2)
    g = GeoJSON(data=feature, hover_style={'fillColor': 'red'})
    g.on_hover(hover_handler)
    m.add_layer(g)

    for place in feature['properties']['places']:
        m.add_layer(place2layer(place))

    clear_output(wait=True)
    display(m, HBox([p, n]), l)

In [11]:
import json
with open('pleiades-bboxes.ndjson') as f:
    features = [json.loads(line) for line in f.readlines()]
    index = 0
    def click_handler(b):
        global index
        if b == p and index > 0:
            index -= 1
        if b == n and index < (len(features) - 1):
            index += 1
        show(features[index])
    p.on_click(click_handler)
    n.on_click(click_handler)

In [12]:
show(features[index])

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

HTML(value='Spatial coverage for Pleiades period "mesolithic-middle-east": Sykamina to Tigris/Diglitus (river)…